In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis, and it is likely that the cheese miners' union will strike due to dissatisfaction with the new president.", additional_kwargs={}, response_metadata={}, id='49470a08-ec86-45e1-ad68-9f7cbb04c13e'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='fc4c02e0-fa24-4485-a964-203594ff076f'),
              AIMessage(content='If I were Lunapolis’ new president, I would respond with a careful, humane, and data-driven plan that prioritizes safety, fairness, and open dialogue while protecting essential cheese production. Here’s how I’d approach it.\n\nImmediate actions\n- Acknowledge concerns publicly and invite the Cheese Miner

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis, and it is likely that the cheese miners' union will strike due to dissatisfaction with the new president.


In [6]:
print(response["messages"][-1].content)

If I were Lunapolis’ new president, I would respond with a careful, humane, and data-driven plan that prioritizes safety, fairness, and open dialogue while protecting essential cheese production. Here’s how I’d approach it.

Immediate actions
- Acknowledge concerns publicly and invite the Cheese Miners’ Union to a formal negotiation table right away.
- Form a dedicated negotiation team that includes union representatives, safety officers, and financial/operations planners. Set a clear timeline and ground rules (e.g., no punitive actions against union activity during talks).
- Establish a joint safety and working-conditions task force to review hazards, especially given the extreme climate (highs around 120C and lows near -100C). Publish an initial safety assessment within 14 days.

Listening and diagnosing
- Conduct structured listening sessions with union leadership and rank-and-file miners to identify the top three to five demands.
- Gather data on wages, benefits, safety improvement

## Trim/delete messages

In [7]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [8]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [9]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='db5b2354-8904-4166-9347-5d38c0607d3b'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='65766397-f1f7-4055-bb61-4aa3d85733cf', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='f57278cc-dd07-4334-9a57-183b82d7abd4'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='07222c11-ed59-4a5a-ae77-0f0d02d49d15', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='91604b2a-5db1-4dd4-a475-47ecbd866bff'),
              AIMessage(content='I can’t read your device’s internal temperature from here. If 

In [10]:
print(response["messages"][-1].content)

I can’t read your device’s internal temperature from here. If you can get a temperature reading from the device itself (in the OS or a diagnostics app), you’ll need to check there. Since you’re seeing a heat issue while it’s powered, here are safe, practical steps:

- If it feels hot to the touch, power it off and unplug it. Let it cool for 15–30 minutes before you try again.
- Check ventilation: make sure vents aren’t blocked, remove any thick cases, and clean out any dust from fans or vents.
- Try a different power adapter and outlet to rule out a charging issue.
- If you can boot into a diagnostic tool or BIOS/UEFI, run any hardware diagnostics to check temperatures and fans.
- If the device still won’t turn on or keeps overheating after cooling, contact support.

If you share the exact device type and model (e.g., laptop/phone/tablet, brand and model), I can give model-specific steps to view temperature readings or run diagnostics.
